# User Manual: Sparse Identification of Induction Motor Nonlinear Dynamics in Unbalanced Conditions

**Table of Contents**
<!-- TOC -->
* [Sparse Identification of Induction Motor Nonlinear Dynamics in Unbalanced Conditions](#sparse-identification-of-induction-motor-nonlinear-dynamics-in-unbalanced-conditions)
  * [Configuration](#configuration)
    * [Choice of Regularization / Optimizer](#choice-of-regularization-/-optimizer)
    * [Choice of Library](#choice-of-library)
  * [Usage](#usage)
    * [1) Data Generation](#1-data-generation)
    * [2) Data Preparation](#2-data-preparation)
    * [3) Optimization of hyperparameters](#3-optimization-of-hyperparameters)
    * [4) Model Identification](#4-model-identification)
    * [5) Model Evaluation](#5-model-evaluation)
  * [Additional Info](#additional-info)
<!-- TOC -->

## Configuration
### Choice of Regularization
The choice of regularisation is important for the model creation. The implemented optimizers are `sr3`, `lasso` or `STLSQ` using l1, l1 and l2 regularisation. The choice of regularisation is randomised for the optuna studies (= hyperparameter selection) which means all optimisers are considered. Adding more optimisers is possible, but requires the following changes: 
- in `train_model_source`, a new optimizer instance needs to be created (be it from pysindy, or via `ps.WrappedOptimizer()` ) and passed to the `ps.SINDy()` model initialisation
- in `optimize_parameters.optuna_search`, the `optimizer_list` should be extended with the corresponding str

`sr3` optimises the function with objective $\dot{X} = \Theta(X)\xi$ as follows (Champion et al. 2020):
$$ ||\dot{X}- \Theta(X)\xi ||^2_2 + \lambda L_{1}(W) + 1/(2\nu) * ||\xi - W||^2_2$$
and thus requires two parameters ($\lambda , \nu$).

`STLSQ` optimises the objective (Brunton et al. 2016) by using hard thresholding on the coefficients:
$$|| \dot{X} - \Theta(X)\xi||^2_2 + \alpha ||\xi||^2_2$$
and requires two parameters ($\alpha,$ threshold).

`Lasso` optimises the objective (see sklearn documentation) as follows
$$ 1/(2n_{\text{samples}}) * ||\dot{X}- \Theta(X)\xi||^2_2 + \alpha ||\xi||_1$$
and requires only one parameter $\alpha$.

### Choice of Library
For SINDy to work, a library of candidate functions must be defined. 
Predefined libraries from pySINDy consist of polynomials or Fourier terms, though pySINDy allows for
custom libraries to be defined and fine-tuned for each input variable. 
Some of our own libraries are predefined in `libs.py` and can be called by the user: `get_custom_library_funcs()`. For the optuna studies (= hyperparameter selection) the libraries are randomly chosen from the `get_library_names()` function inside `libs.py`. Note that optuna does not allow a dynamical search space for a categorial; thus changing the library list requires a new optuna study. 

## Usage

In [1]:
%matplotlib QtAgg
import matplotlib.pyplot as plt

import os
from optimize_parameters import parameter_search, optimize_parameters, plot_optuna_data
from source import *
from train_model_source import make_model, simulate_model

### 1) Data Generation
The data generation is completely distict from the other aspects of the project. 
It uses the time-stepping solver IMMEC to model the induction machine and obtain the data.
The user can generate the data by running the script `data_generation.py` and setting the parameters in the script.
The user can choose to generate test or training data, resulting in one or multiple (`numbr_of_simulations`) 
simulations respectively. An initial eccentricity can also be set by `ecc_value` (value between 0 and 1) and `ecc_dir` 
(the direction in $x, y$ coordinates). The mode can be set to `linear`or `nonlinear` to simulate the machine.

The function automatically creates a folder inside the `train-data` or `test-data` folder with the name of the date, 
and saves the simulation data in a `.npz`-file with the name provided by the user by `save_name`.

The `.npz`-file contains the following arrays:
- `i_st` - Stator current
- `omega_rot` - Rotor angular speed
- `T_em` - Electromagnetic torque
- `F_em` - Electromagnetic force or UMP
- `v_applied` - Applied line voltages
- `T_l` - Load torque
- `ecc` - Eccentricity
- `time` - Simulated time
- `flux_st_yoke` - Stator yoke flux
- `gamma_rot` - Rotor angle
- `wcoe` - Magnetic coenergy

If multiple simulations are saved in one file, which is the case for traindata, 
the arrays have an additional (3rd) dimension for each simulation.

In [2]:
### DATA TRAINING FILES
path_to_data_files = os.path.join(os.getcwd(), 'train-data', '07-29-default', 'IMMEC_0ecc_5.0sec.npz')

### DATA TEST FILES
path_to_test_file = os.path.join(os.getcwd(), 'test-data', '07-29-default', 'IMMEC_0ecc_5.0sec.npz')

In [3]:
### Visualise the data
plot_immec_data(path_to_data_files, simulation_number = 10, title= "Train")

d_air hardcoded for Cantoni


In [7]:
plot_immec_data(path_to_test_file, title="Test")

d_air hardcoded for Cantoni


### 2) Data Preparation
In order to create a model, the training data must be prepared. This is done in the script `data_preparation.py`, which is 
called by the  `prepare_data()` function during training and postprocessing by other scripts. The user can call this function if desired,
but this is not necessary. The function takes the following arguments:
- `path_to_data_file` - Path to the `.npz`-file containing the data
- `test_data` - default False, this omits the extra preparation needed for trainingdata
- `number_of_trainfiles` - default -1 (all files), can be set to a number if not all simulations should be considered. The choice of selected simulations is random. This can be useful to reduce the training samples for large datasets.
- `use_estimate_for_v` - default False, if True, the `v_abc` are estimated from the line voltages.
- `usage_per_trainfile` - default 0.5, the percentage of the data used from each simulation.
- `ecc_input`- default False, if True, the eccentricity is used as an input variable to the model.

The function returns a dictionary containing the following arrays:
- `x`- Currents
- `u`- Input values, if `ecc_input` is True, the eccentricity is also included
- `xdot` - Time derivative of the currents
- `feature_names` - Names of the features to pass to the SINDy model

Additionally, as one might want to fit a SINDy model for the torque or UMP (by replacing `xdot`), the following are also present:
- `UMP` - Unbalanced magnetic pull
- `T_em` - Electromagnetic torque
- `wcoe` - The magentic coenergy



If the data is trainingsdata, it is split up into train and validation data (80% - 20%), in which case the dictionary also contains all the previous values but ending with `_train` and `_val`. Note that for model creation, the torque or UMP can be passed through `xdot` for solving e.g. 
$$ T = f(x,u) $$ instead of $\dot{x} = f(x,u) $.

### 3) Optimization of hyperparameters
Assuming the data generation is done, the user can start the optimization of the hyperparameters. 

As described in [this section](#choice-of-regularisation), the Lasso and SR3 regulators are considered, 
yielding 1 and 2 hyperparameters respectively. Hence, the validation data is used to select the best parameter values. 
This can be combined with various selections of library candidate functions, enlarging the search-space.

For this purpose, a Python package called `Optuna` is used, which searches for the pareto-optimal solution. 
The user can initialise a study by calling the function `optimize_parameters()` from the script `optimize_parameters.py`, by which a study is created.
The function requires the following arguments:
- `path_to_data_files` - Path to the `.npz`-file containing the training data
- `mode`- default 'torque', can be set to 'ump', 'currents' or 'wcoe', specifying what the model predicts
- `additional_name` - default None, a string that is added to the name of the study
- `n_jobs` - default 1, number of cores to be used for the optimization, to run in paralell
- `n_trials` - default 100, number of trials to be performed per core

The ranges of the parameters are predefined inside the function, but can be changed by the user. 
The library candidate functions are called from `libs.py` by the function `get_library_names()` during the search. 
The user should set the desired libraries by changing the returned values of this function. 

The resulting study is saved in the `optuna_studies` folder, which can be accessed by calling the `plot_optuna_data()` function from the same script.


In [3]:
### Optimising parameters takes a long time to run
optimize_parameters(path_to_data_files, 
                    mode="torque", 
                    additional_name="_jupyter",
                    n_jobs = 1, n_trials = 1,
                    ecc_input=False)
# generates only one trial, usually 1000 is more desired

ecc_input = False
Loading data
Done loading data
Calculating xdots
Assume all t_vec are equal for all simulations
Done calculating xdots
time trim:  0.5


[I 2024-08-19 18:31:08,358] A new study created in RDB with name: optuna_studies//torque_jupyter-optuna-study
[I 2024-08-19 18:31:08,417] Using an existing study with name 'optuna_studies//torque_jupyter-optuna-study' instead of creating a new one.
C:\Users\emmav\anaconda3\envs\immec_env\Lib\site-packages\pysindy\utils\axes.py:126: AxesWarning: 2 axes labeled for array with 1 axes
  warnings.warn(
C:\Users\emmav\anaconda3\envs\immec_env\Lib\site-packages\pysindy\utils\axes.py:126: AxesWarning: 2 axes labeled for array with 1 axes
  warnings.warn(
[I 2024-08-19 18:31:17,768] Trial 0 finished with values: [0.021982061753283082, 35.0] and parameters: {'lib_choice': 'linear-specific', 'optimizer': 'lasso', 'alphas': 1.3431689039625701e-05}. 


In [4]:
### Plot the study
plot_optuna_data('torque_jupyter-optuna-study')
### or a premade study
plot_optuna_data('torquelinear_premade-optuna-study')

[I 2024-08-23 11:33:27,703] Study name was omitted but trying to load 'optuna_studies//torque_jupyter-optuna-study' because that was the only study found in the storage.


['optuna_studies//torque_jupyter-optuna-study']
Trial count: 1


[I 2024-08-23 11:33:29,328] Study name was omitted but trying to load 'optuna_studies//torquelinear-optuna-study' because that was the only study found in the storage.


['optuna_studies//torquelinear-optuna-study']
Trial count: 1000


### 4) Model Identification
Now the desired hyperparameters and optimiser are known, the user can start the model identification. 
This is done by calling the function `make_model()` from the script `train_model_source.py`. The function requires the following arguments:
- `path_to_data_files` - Path to the `.npz`-file containing the training data
- `modeltype`- Can be set to 'torque', 'ump', 'torque-ump', 'currents' or 'wcoe', specifying what the model predicts
- `optimizer`- Either 'lasso' or 'sr3', specifying the regularisation method
- `lib` - The chosen library candidate functions
- `nmbr_of_train` - default -1 (all files), can be set to a number if not all simulations should be considered.
- `alpha` - default None, the regularisation parameter for lasso
- `nu` - default None, the first regularisation parameter for sr3
- `lamb` - default None, the second regularisation parameter for sr3
- `model_name` - default None, a string that is added to the name of the model

When a model is created, it is saved as a `.pkl`-file in the `models` folder. The model can be loaded by calling the `load_model()` function from `source.py`.

Note that the `AxesWarning` comes from pysindy and cannot be turned off. The error has to do with the one-dimensional vector (Torque or W_co) having the shape (n,) instead of (n,1). 

In [9]:
saved_model = make_model(path_to_data_files, modeltype='torque', optimizer='sr3',
                   nmbr_of_train=-1, lib='poly_2nd_order', nu=1.978e-10, lamb=5.3e-9,
                   modelname='jupyter_example')
print(saved_model)

ecc input: True
Loading data
Done loading data
Calculating xdots
Assume all t_vec are equal for all simulations
Done calculating xdots
time trim:  0.5
Zero ecc, not added to input data
SR3_L1 optimisation
Fitting model


C:\Users\emmav\anaconda3\envs\immec_env\Lib\site-packages\pysindy\utils\axes.py:126: AxesWarning:

2 axes labeled for array with 1 axes



(i_d)' = -0.0006291776 1 + -0.1338899272 i_d + 0.1904517492 i_q + 1.5241252377 v_d + 0.8603132026 v_q + 0.0005917922 v_0 + -1.7794931954 I_d + 1.2676031916 I_q + 0.1395307204 V_d + -0.2623416482 V_q + -0.0160969501 V_0 + -0.0071486507 \omega + 0.0000310678 f + -0.0002912295 i_d^2 + -0.0005683878 i_d i_q + 0.0000850227 i_d v_d + 0.0000567212 i_d v_q + -0.0000212265 i_d v_0 + -0.3443391978 i_d I_d + 6.7397911456 i_d I_q + 0.0606778720 i_d V_d + -0.9235090793 i_d V_q + 0.0067666844 i_d V_0 + -0.0000071440 i_d \omega + 0.0000213525 i_d f + -0.0002995962 i_q^2 + -0.0000489452 i_q v_d + 0.0000925376 i_q v_q + -0.0000062224 i_q v_0 + -6.7500520176 i_q I_d + -0.4625782648 i_q I_q + 0.9214469685 i_q V_d + 0.0620356447 i_q V_q + -0.0177279593 i_q V_0 + -0.0000170423 i_q \omega + 0.0000996364 i_q f + -0.0000074655 v_d^2 + -0.0000000570 v_d v_q + 0.0000000209 v_d v_0 + 0.0253811115 v_d I_d + -0.0484512414 v_d I_q + -1.3773275079 v_d V_d + -0.0049801819 v_d V_q + -0.0001330742 v_d V_0 + 0.000000025

C:\Users\emmav\anaconda3\envs\immec_env\Lib\site-packages\pysindy\utils\axes.py:126: AxesWarning:

2 axes labeled for array with 1 axes



MSE: 2.3046952778851247e-06
Saving model


### 5) Model Evaluation
Now, the model's performance can be evaluated on the test data. This is done by calling the function `simulate_model()`
from the script `train_model_source.py`. The function requires the following arguments:
- `model_name` - The name of the model from the `models` folder
- `path_to_test_file` - Path to the `.npz`-file containing the test data
- `modeltype`- Can be set to 'torque', 'ump', 'torque-ump', 'currents' or 'wcoe', specifying what the model predicts
- `do_time_simulation` - default False, only relevant if `modeltype` == 'currents'. Then the `xdot` is solved by `solve_ivp` 
to retrieve the prediction of `x`

This function returns the predicted and expected values. These can also be plotted in the frequency domain by the `plot_fourier()` function from `source.py`.

In [11]:
model = saved_model
pred, test = simulate_model(model, path_to_test_file, modeltype="torque", do_time_simulation=False, show=True)

(i_d)' = -0.001 1 + -0.134 i_d + 0.190 i_q + 1.524 v_d + 0.860 v_q + 0.001 v_0 + -1.779 I_d + 1.268 I_q + 0.140 V_d + -0.262 V_q + -0.016 V_0 + -0.007 \omega + -0.001 i_d i_q + -0.344 i_d I_d + 6.740 i_d I_q + 0.061 i_d V_d + -0.924 i_d V_q + 0.007 i_d V_0 + -6.750 i_q I_d + -0.463 i_q I_q + 0.921 i_q V_d + 0.062 i_q V_q + -0.018 i_q V_0 + 0.025 v_d I_d + -0.048 v_d I_q + -1.377 v_d V_d + -0.005 v_d V_q + 0.050 v_q I_d + 0.026 v_q I_q + 0.005 v_q V_d + -1.377 v_q V_q + -0.005 v_0 I_q + -0.008 v_0 V_0 + 1.745 I_d^2 + -0.230 I_d I_q + 0.247 I_d V_d + 1.234 I_d V_q + -0.003 I_d V_0 + -0.001 I_d \omega + -0.590 I_q^2 + -1.330 I_q V_d + 0.571 I_q V_q + -0.002 I_q V_0 + -0.013 I_q \omega + 0.003 I_q f + -0.012 V_d^2 + 0.007 V_d V_q + -0.019 V_d V_0 + -0.001 V_d \omega + -0.014 V_q^2 + -0.021 V_q V_0 + 0.002 V_q \omega + -0.001 V_0 \omega + 0.005 V_0 f
ecc input:  False
Loading data
Done loading data
Calculating xdots
Assume all t_vec are equal for all simulations
Done calculating xdots
MSE o

## Additional Information
The results from this summer internship are present in this git repository, and can be reproduced using the following scripts:

- `run_optuna_studies.py` (not recommended to run, creates 16x1000 models and requires a lot of memory + time)
- `model_creation_and_post_process.ipynb` notebook for visualising the optuna studies, creating and testing (pareto-optimal) models
- For creating the new figures (e.g. optuna studies via matplotlib), one should check the `temp_optuna_plot.py` script.